In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [3]:
importlib.reload(overview)
file_name = "proAnubis_240815_1759.raw"
chunks, times, fReader = overview.get_chunks(file_name, max_process_event=1_000_000)

Initial time 2024-08-15 16:59:49.312501 1723737589.312501


Processing Chunks proAnubis_240815_1759.raw:  50%|████▉     | 4996/10000 [00:53<00:40, 124.73Chunks/s]

Event time: 2024-08-15 17:01:26.654146


Processing Chunks proAnubis_240815_1759.raw:  50%|█████     | 5019/10000 [01:00<09:08,  9.08Chunks/s] 

Saved


Processing Chunks proAnubis_240815_1759.raw: 100%|█████████▉| 9991/10000 [01:53<00:00, 127.14Chunks/s]

Event time: 2024-08-15 17:03:04.904445


Processing Chunks proAnubis_240815_1759.raw: 100%|██████████| 10000/10000 [02:06<00:00, 79.11Chunks/s]

Saved
Ending time: 2024-08-15 17:03:04.904445
Number of chunks: 10000


In [11]:
#find events with multiple clusters
importlib.reload(RTools)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(chunks[0], 0)
for processedEvents, event_chunk in enumerate(chunks[1:]):

      reconstructor.update_event(event_chunk, processedEvents)
      # populate_hits turns TDC bit wise information into their corresponding strips
      reconstructor.populate_hits()
      # This is optionnal, and requires the residual of eta and phi
         # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
      clusters = reconstructor.make_cluster()
      for evt_num, cluster in enumerate(clusters):
         repeated = 0
         for rpc in range(6):
            if len(cluster[2][rpc][0]) == 2 and len(cluster[2][rpc][1]) == 2: #phi and eta
               repeated += 1

         if repeated == 6:
            #heatmap
            hits = []
            for rpc in range(6):
               hits.extend([phi for phi in cluster[2][rpc][0]])
               hits.extend([eta for eta in cluster[2][rpc][1]])
            hits = [h[0] for h in hits]
            heatMaps = [np.zeros((32,64)) for rpc in range(6)]
            for hit in hits:
               if hit[3]:
                  heatMaps[hit[0]][hit[1],:] += np.ones(64)
               else:
                  heatMaps[hit[0]][:,hit[1]] += np.ones(32)
            #reconstruction
            path = RTools.reconstruct_timed_Chi2_ByRPC(cluster, 3, -1)
            if path:
               print("Good")
               centroid, direction, opt_coords, _, chi2, dT, _, _ = path
               for rpc, point in enumerate(opt_coords):
                  try:
                     heatMaps[rpc][31 - int(point[1]/ 2.9844)][int(point[0]/ 2.7625)] += 3
                  except:
                     print((int(point[0]/ 2.7625), int(point[1]/ 2.9844)))
            for rpc in range(6):
                plt.imshow(heatMaps[rpc], interpolation='nearest')
                plt.title(f'Event {evt_num} RPC {rpc}')
                plt.savefig(f'video/images_video/double{processedEvents*100+10*evt_num+rpc}.png')
            
      #print("Clust")

Good
Good
Good
Good
Good
Good
Good
Good
